In [21]:
import numpy as np
import pickle
import pandas as pd
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from tqdm.notebook import tqdm

In [22]:
stopwords = ["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", 
             "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", 
             "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", 
             "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", 
             "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", 
             "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", 
             "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", 
             "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", 
             "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", 
             "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", 
             "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", 
             "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to",
             "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren",
             "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won",
             "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", 
             "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", 
             "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", 
             "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", 
             "would"]

In [24]:
yle_data = pd.read_csv('memad/captions_clean-_ureilhutu.csv')
yle_data.head()

,Unnamed: 0,caption,video_id,video,frame
0,0,a man is dancing and singing in a room,202000823730_9,202000823730,9
1,1,a pair of skis in the air with a pair of scissors,202000823730_137,202000823730,137
2,2,a man is holding a tennis racket in his hand,202000823730_451,202000823730,451
3,3,a man is jumping and spinning in circles,202000823730_725,202000823730,725
4,4,a man is jumping and spinning in circles,202000823730_860,202000823730,860


In [25]:
surrey_data = pd.read_csv('memad/Surrey_captions_clean-1.csv')
surrey_data.head()

,Unnamed: 0,caption,video_id,video,frame
0,0,car is driving down a road,000200_23,200,23
1,1,man is walking down a street and then he stops,000200_83,200,83
2,2,car is driving down a road,000200_178,200,178
3,3,man is driving a car and talking to someone,000200_261,200,261
4,4,woman with long hair is smiling and talking,000200_324,200,324


In [27]:
w2v = pickle.load(open('../conceptnet/glove.6B/glove.w2v.6B.300d.pickle', 'rb'))

In [28]:
X_w2v   = []

for i, entry in tqdm(yle_data.iterrows(), total=len(yle_data)):
    text = tokenize(entry['caption'])
    
    words = [word for word in text.split(' ') if word in w2v]
    x_w2v = np.zeros([300]) if not words else np.mean([w2v[word] for word in words], axis=0)
    
    X_w2v.append(x_w2v)

In [33]:
model_st = pickle.load(open('me20_svr_w2v_st_model.pickle', 'rb'))
model_lt = pickle.load(open('me20_svr_w2v_lt_model.pickle', 'rb'))

In [34]:
y_st_pred = model_st.predict(X_w2v)
y_lt_pred = model_lt.predict(X_w2v)

In [36]:
yle_data['results_st'] = y_st_pred
yle_data['results_lt'] = y_lt_pred

In [38]:
yle_data[['caption', 'video_id', 'video', 'frame', 'results_st', 'results_lt']].head()

,caption,video_id,video,frame,results_st,results_lt
0,a man is dancing and singing in a room,202000823730_9,202000823730,9,0.829784,0.780054
1,a pair of skis in the air with a pair of scissors,202000823730_137,202000823730,137,0.830087,0.780025
2,a man is holding a tennis racket in his hand,202000823730_451,202000823730,451,0.829856,0.779800
3,a man is jumping and spinning in circles,202000823730_725,202000823730,725,0.830050,0.780002
4,a man is jumping and spinning in circles,202000823730_860,202000823730,860,0.830050,0.780002


In [39]:
yle_data[['caption', 'video_id', 'video', 'frame', 'results_st', 'results_lt']].to_csv('memad/urheiluruutu_results.csv')

In [41]:
X_w2v   = []

for i, entry in tqdm(surrey_data.iterrows(), total=len(surrey_data)):
    text = tokenize(entry['caption'])
    
    words = [word for word in text.split(' ') if word in w2v]
    x_w2v = np.zeros([300]) if not words else np.mean([w2v[word] for word in words], axis=0)
    
    X_w2v.append(x_w2v)

In [42]:
y_st_pred = model_st.predict(X_w2v)
y_lt_pred = model_lt.predict(X_w2v)

In [43]:
surrey_data['results_st'] = y_st_pred
surrey_data['results_lt'] = y_lt_pred

In [44]:
surrey_data[['caption', 'video_id', 'video', 'frame', 'results_st', 'results_lt']].head()

,caption,video_id,video,frame,results_st,results_lt
0,car is driving down a road,000200_23,200,23,0.830093,0.779982
1,man is walking down a street and then he stops,000200_83,200,83,0.829837,0.779954
2,car is driving down a road,000200_178,200,178,0.830093,0.779982
3,man is driving a car and talking to someone,000200_261,200,261,0.829979,0.779942
4,woman with long hair is smiling and talking,000200_324,200,324,0.829920,0.780008


In [45]:
surrey_data[['caption', 'video_id', 'video', 'frame', 'results_st', 'results_lt']].to_csv('memad/urheiluruutu_results.csv')